In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: trainingSet/trainingSet/8/img_4000.jpg  
  inflating: trainingSet/trainingSet/8/img_40004.jpg  
  inflating: trainingSet/trainingSet/8/img_4001.jpg  
  inflating: trainingSet/trainingSet/8/img_40019.jpg  
  inflating: trainingSet/trainingSet/8/img_40026.jpg  
  inflating: trainingSet/trainingSet/8/img_40029.jpg  
  inflating: trainingSet/trainingSet/8/img_40032.jpg  
  inflating: trainingSet/trainingSet/8/img_40055.jpg  
  inflating: trainingSet/trainingSet/8/img_40065.jpg  
  inflating: trainingSet/trainingSet/8/img_4007.jpg  
  inflating: trainingSet/trainingSet/8/img_40109.jpg  
  inflating: trainingSet/trainingSet/8/img_40120.jpg  
  inflating: trainingSet/trainingSet/8/img_40133.jpg  
  inflating: trainingSet/trainingSet/8/img_40136.jpg  
  inflating: trainingSet/trainingSet/8/img_40150.jpg  
  inflating: trainingSet/trainingSet/8/img_40157.jpg  
  inflating: trainingSet/trainingSet/8/img_40173.jpg  
  inflating: trai

In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [ ]:
train_path = "/content/trainingSet/trainingSet/"

In [ ]:
img_path = list(paths.list_images(train_path))

In [ ]:
data = []
label = []
for (i, path) in enumerate(img_path):
  img = cv2.imread(path,  cv2.IMREAD_GRAYSCALE)
  img = np.array(img).flatten()
  lab = path.split(os.path.sep)[-2]
  data.append(img/255)
  label.append(lab)
  print(i)

Streaming output truncated to the last 5000 lines.
37000
37001
37002
37003
37004
37005
37006
37007
37008
37009
37010
37011
37012
37013
37014
37015
37016
37017
37018
37019
37020
37021
37022
37023
37024
37025
37026
37027
37028
37029
37030
37031
37032
37033
37034
37035
37036
37037
37038
37039
37040
37041
37042
37043
37044
37045
37046
37047
37048
37049
37050
37051
37052
37053
37054
37055
37056
37057
37058
37059
37060
37061
37062
37063
37064
37065
37066
37067
37068
37069
37070
37071
37072
37073
37074
37075
37076
37077
37078
37079
37080
37081
37082
37083
37084
37085
37086
37087
37088
37089
37090
37091
37092
37093
37094
37095
37096
37097
37098
37099
37100
37101
37102
37103
37104
37105
37106
37107
37108
37109
37110
37111
37112
37113
37114
37115
37116
37117
37118
37119
37120
37121
37122
37123
37124
37125
37126
37127
37128
37129
37130
37131
37132
37133
37134
37135
37136
37137
37138
37139
37140
37141
37142
37143
37144
37145
37146
37147
37148
37149
37150
37151
37152
37153
37154
37155
37156
37157
3

In [ ]:
lb = LabelBinarizer()
label_list = lb.fit_transform(label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    label_list, 
                                                    test_size=0.1, 
                                                    random_state=42)

In [ ]:
def make_clients(img, label, num_clients = 10):
  client_names = ["client_"+str(i) for i in range(num_clients)]
  data = list(zip(img, label))
  random.shuffle(data)
  size = len(data)//num_clients
  split_data = [data[i:i+size] for i in range(0, size*num_clients, size)]
  client_data = {client_names[i]: split_data[i] for i in range(len(client_names))}
  return client_data

In [ ]:
client_data = make_clients(X_train, y_train)

In [ ]:
def batch(zip_data, batch_size = 32):
  data, label = zip(*zip_data)
  dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
  return dataset.shuffle(len(label)).batch(batch_size)

In [ ]:
client_batched = {}
for (name, data) in client_data.items():
  client_batched[name] = batch(data)

test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [ ]:
class MLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [ ]:
model = MLP()
model = model.build(784,10)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               157000    
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 199,210
Trainable params: 199,210
Non-trainable params: 0
__________________________________________________

In [ ]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )   

In [ ]:
weight_scale = []
for (name, data) in client_data.items():
  weight_scale.append(len(data)/len(X_train))
weight_scale = np.array(weight_scale).reshape((len(weight_scale), 1))

In [ ]:
alpha = []
for (name, data) in client_data.items():
  alpha.append(0.5)

In [ ]:
gamma = np.random.dirichlet(alpha)
gamma = np.array(gamma).reshape((len(gamma), 1))

In [ ]:
np.dot(gamma.T,weight_scale)

array([[0.1]])

In [ ]:
gamma*weight_scale

array([[0.00094591],
       [0.00200403],
       [0.00098766],
       [0.00656167],
       [0.05778708],
       [0.00249847],
       [0.00406889],
       [0.00062795],
       [0.0243611 ],
       [0.00015723]])

In [ ]:
def sum_scaled_weights_direc(scaled_weight_list):
    avg_grad = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

def sum_scaled_weights(scaled_weight_list):
    #print(type(scaled_weight_list[0][0]))
    avg_grad = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        #print(np.shape(grad_list_tuple))
        if len(np.shape(grad_list_tuple)) == 3:
          w = weight_scale.reshape((len(weight_scale), 1, 1))
        else:
          w = weight_scale
        layer_mean = np.sum(grad_list_tuple*w, axis=0)
        #print(np.shape(layer_mean))
        layer_variance = np.sum(np.power(grad_list_tuple - layer_mean, 2)*w, axis = 0) 
        avg_grad.append(np.random.normal(layer_mean, layer_variance))
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
np.random.normal([[3,4],[4,5]], [[1,1],[1,0]])

array([[1.24015813, 4.59290974],
       [3.55066984, 5.        ]])

In [ ]:
mlp_global = MLP()
global_model = mlp_global.build(784, 10)

for comm_round in range(comms_round):
            
    
    global_weights = global_model.get_weights()
    
    
    scaled_local_weight_list = list()
    client_names = list(client_batched.keys())
    random.shuffle(client_names)
    
    for client in client_names:
        mlp_local = MLP()
        local_model = mlp_local.build(784, 10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        local_model.set_weights(global_weights)
        
        local_model.fit(client_batched[client], epochs=1, verbose=0)
        scaled_local_weight_list.append(local_model.get_weights())
        
        K.clear_session()
    constant = np.dot(gamma.T,weight_scale)
    mult_factor = gamma*weight_scale
    mult_factor = mult_factor/constant

    average_weights = sum_scaled_weights_direc(scaled_local_weight_list*mult_factor)
    global_model.set_weights(average_weights)

    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


comm_round: 0 | global_acc: 87.905% | global_loss: 1.6734099388122559
comm_round: 1 | global_acc: 90.667% | global_loss: 1.6216264963150024
comm_round: 2 | global_acc: 91.929% | global_loss: 1.5973949432373047
comm_round: 3 | global_acc: 92.190% | global_loss: 1.5856961011886597
comm_round: 4 | global_acc: 93.167% | global_loss: 1.5778454542160034
comm_round: 5 | global_acc: 93.548% | global_loss: 1.5696663856506348
comm_round: 6 | global_acc: 93.881% | global_loss: 1.563966989517212
comm_round: 7 | global_acc: 94.119% | global_loss: 1.5600073337554932
comm_round: 8 | global_acc: 94.071% | global_loss: 1.5563687086105347
comm_round: 9 | global_acc: 94.286% | global_loss: 1.5519994497299194
comm_round: 10 | global_acc: 94.476% | global_loss: 1.549091100692749
comm_round: 11 | global_acc: 94.881% | global_loss: 1.5471138954162598
comm_round: 12 | global_acc: 94.714% | global_loss: 1.5450533628463745
comm_round: 13 | global_acc: 94.881% | global_loss: 1.5424789190292358
comm_round: 14 | g

In [ ]:
from random import randrange
for i in range(50):
  a = str(randrange(10))
  b = str(randrange(10))
  c = str(randrange(10))
  print(a + ' ' + b + ' ' + c)

9 3 4
2 8 0
9 9 3
8 5 6
8 1 6
2 3 8
4 0 1
3 7 6
7 7 6
6 9 6
8 4 5
2 4 2
3 4 7
8 4 6
2 8 3
3 1 9
7 1 3
5 0 2
3 3 7
7 9 0
2 2 2
4 3 1
1 5 9
0 0 2
9 1 9
9 3 3
0 8 3
0 7 9
2 9 7
3 5 4
0 9 2
1 0 0
4 8 4
5 7 8
3 8 4
2 5 5
8 4 9
8 3 0
1 6 7
0 7 2
6 1 5
1 3 9
4 2 4
8 5 1
6 7 0
2 7 7
8 2 4
4 9 8
7 1 0
7 9 4
